## 1. Setting up the Environment

#### 1.1. Install AML SDK

PYPI reference: https://pypi.org/project/azureml-core/

Using !pip for demonstration purposes only. Use the terminal and managed conda/python environments for this! 

In [ ]:
!pip install azureml-core

#### 1.2. Checking the version

In [ ]:
!pip show azureml-core

AML Python SDK Installation guide: https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py

---

## 2. Create AML Workspace

#### 2.1. Import the necessary modules to connect to the workspace

In [ ]:
# import the SDK modules to manage workspace
from azureml.core import Workspace

#### 2.2. Create AML Workspace

In [ ]:
# define workspace info
workspace_name      = 'aml'
subscription_id     = '##YOUR SUBSCRIPTION KEY##'
resource_group_name = 'SvenskaHandelsbanken'
service_location    = 'westeurope'

In [ ]:
# Verify that workspace does not exist already
try: 
    # try to get the workspace
    ws_obj = Workspace.get(name = workspace_name, subscription_id = subscription_id, resource_group = resource_group_name)
    print('Found existing workspace.')

except:
    # create the Azure Machine Learning Workspace into the specified subscription, resource group, and region
    # this command creates also a storage account, application insights, and key vault
    ws_obj = Workspace.create(name            = workspace_name,
                              subscription_id = subscription_id,
                              resource_group  = resource_group_name,
                              location        = service_location)

#### 2.3. List all the workspaces

In [ ]:
# eventually you can get all the workspaces from a subscription or resource group
Workspace.list(subscription_id = subscription_id, resource_group = resource_group_name)

---

## 3. Create Remote Compute

#### 3.1. Import the necessary modules to control the compute

In [ ]:
# import the SDK modules to manage computes
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [ ]:
# define compute cluster info
cpu_cluster_name = 'cpucluster'

#### 3.2. Provision the compute targets

In [ ]:
# Verify that cluster does not exist already
try:
    # try to get the compute cluster
    cpu_cluster = ComputeTarget(workspace = ws_obj, name = cpu_cluster_name)
    print('Found existing cluster')
    
except ComputeTargetException:
    # create the compute cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws_obj, cpu_cluster_name, compute_config)

# check the status of the provisioning
cpu_cluster.wait_for_completion(show_output=True)

---

## 4. Prepare the Execution

#### 4.1. Import the necessary modules to control the execution

In [ ]:
# import the SDK modules to manage runs
from azureml.core import Experiment, Environment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies

#### 4.2. Prepare the config of the run and the parametrization of the script

In [ ]:
# define run info 
experiment_name  = 'monte_carlo_simulation'
curated_env_name = 'AzureML-Minimal'
env_name         = 'mc_env'
source_dir       = './scripts/'
script_name      = 'mc_simulations.py'

In [ ]:
# define the arguments 
num_simulations  = 1000
avg = 1
std_dev = .1
num_reps = 500

list_args = ['--num_simulations', num_simulations,
             '--avg', avg,
             '--std_dev', std_dev,
             '--num_reps', num_reps]

#### 4.3. Prepare the Experiment Object

In [ ]:
experiment_obj = Experiment(workspace = ws_obj, name = experiment_name)

#### 4.4. Prepare the environmnet

In [ ]:
curated_env_obj = Environment.get(workspace = ws_obj, name = curated_env_name)
env_obj = curated_env_obj.clone(env_name)

In [ ]:
conda_dep = CondaDependencies()
conda_dep.add_conda_package('numpy')
conda_dep.add_conda_package('pandas')

# Adds dependencies to PythonSection of env_obj
env_obj.python.conda_dependencies = conda_dep

#### 4.5. Bind everything into the Run Config

In [ ]:
script_run_config = ScriptRunConfig(source_directory = source_dir,
                                    script           = script_name,
                                    arguments        = list_args,
                                    compute_target   = cpu_cluster,
                                    environment      = env_obj)

---

## 5. Run

#### 5.1. Run the experiment with the specified configuration

In [ ]:
# Start the asynchronous execution
run_obj = experiment_obj.submit(config = script_run_config)

#### 5.2. Manage the execution

In [ ]:
# Eventually, you can lock the execution until it has been completed
run_obj.wait_for_completion(show_output = True)

In [ ]:
# Or just check the status
run_obj.get_status()